# Practice Questions Solutions

# Retrieval For Images

![Alt](diagrams/RAG%20-%20Retrieval%20For%20Images.jpg)

### Practice 
1) Use FAISS library + an embedding model for vision
2) Download some cat/dog images from google and store them in images directory
3) Create an image store and VectorDB, store them in images directory
4) Perform similarity search and retrieve top 2 images

In [ ]:
# Write code

## Cross-model/Mulitmodal Retrieval

![Alt text](diagrams/RAG%20-%20Cross-Model%20RetrievalMultimodal%20Retrieval.jpg)

## Text-to-Image Retrieval

![Alt text](diagrams/RAG%20-%20Text-to-Image%20Retrieval.jpg)

### Practice 
1) Use FAISS library & CLIP's vision encoder + text decoder
2) Load the existing image store + VectorDB created previously
3) write a sample query
3) Perform similarity search and retrieve top 2 images

In [ ]:
# Write code

## Image-to-Text Retreival

![Alt text](diagrams/RAG%20-%20Image-to-text%20Retrieval.jpg)

### Practice 
1) Use FAISS library & CLIP's vision encoder + text decoder.
2) Write some sentences in a sentences.txt file and store it in texts directory.
3) load this sentences.txt, make document store + VectorDB out of it.
4) Also store document store + VectorDB in texts folder. 
3) Use a sample image from images folder as query.
3) Perform similarity search and retrieve top 2 sentences.

In [ ]:
# Write code